## Generate the skip trigram dataset


In [13]:
from transformer_lens import HookedTransformer, HookedTransformerConfig
import torch

num_digits = 9
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cfg = HookedTransformerConfig(
    n_layers = 1,
    d_model  = 128,
    d_head   = 32,
    n_heads  = 4,
    d_vocab  = num_digits,
    n_ctx    = 128,
    attn_only= True,
    device   = device,
)
model = HookedTransformer(cfg)
from transformer_lens.components import Unembed, Embed

# Override unembeddings and embeddings to be identity
embed = Embed(cfg)
embed.W_E = torch.nn.Parameter(torch.eye(cfg.d_vocab), requires_grad=False)
unembed = Unembed(cfg)
unembed.W_U = torch.nn.Parameter(torch.eye(cfg.d_vocab), requires_grad=False)
unembed.b_U = torch.nn.Parameter(torch.zeros(cfg.d_vocab), requires_grad=False)

model.embed = embed
model.unembed = unembed

model = model.to(device)

Moving model to device:  cuda


In [15]:
inp = torch.randint(0, 10, (16, 10), device=device)

print(inp.shape)
print(model(inp))

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
